In [1]:
pip install torch==2.4.1 torchvision torchaudio


ERROR: Could not find a version that satisfies the requirement torch==2.4.1 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2)
ERROR: No matching distribution found for torch==2.4.1
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
from langchain.chat_models import init_chat_model
from langchain_huggingface import HuggingFaceEmbeddings\
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone

In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""

model = init_chat_model(
    "microsoft/Phi-3-mini-4k-instruct",
    model_provider="huggingface",
    temperature=0.7,
    max_tokens=1024,
)

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

pc = Pinecone(api_key='')
index = pc.Index(index_name)

vector_store = PineconeVectorStore(embedding=embeddings, index=index)


In [ ]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("my_notes.txt")
docs = loader.load()

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  
    chunk_overlap=200, 
    add_start_index=True,  
all_splits = text_splitter.split_documents(docs)


document_ids = vector_store.add_documents(documents=all_splits)


In [ ]:
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [ ]:
from langchain.agents import create_agent


tools = [retrieve_context]
prompt = (
    "You have access to a text document. "
    "Use the tool to help answer user queries."
)
agent = create_agent(model, tools, system_prompt=prompt)